In [1]:
from __future__ import print_function
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

from tensorflow.keras.optimizers import RMSprop

from nltk import word_tokenize
from nltk.stem.porter import PorterStemmer
import nltk
from nltk.corpus import stopwords
import string
import numpy as np
import pandas as pd

import os
import sys
from keras.preprocessing.text import Tokenizer

from tensorflow.keras.preprocessing.sequence import pad_sequences
import keras.utils as ku
import keras.layers as kl
from tensorflow.keras.layers import Dense, Input, GlobalMaxPooling1D, Concatenate
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Embedding
from tensorflow.keras.models import Model
from keras.initializers import Constant
from tensorflow.keras.preprocessing.sequence import pad_sequences


from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, load_model

#from keras.models import Sequential
from tensorflow.keras.layers import Embedding, Activation, Dense, LSTM, Conv2D, Reshape, dot, Lambda, Dropout, MaxPooling2D
from keras.optimizers import SGD
from itertools import islice
from tensorflow.keras.layers import Flatten
from keras.models import model_from_json
from keras_preprocessing.image import ImageDataGenerator
from keras_preprocessing.sequence import TimeseriesGenerator


from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
import math
import random
from keras.callbacks import TensorBoard, ModelCheckpoint, LearningRateScheduler, EarlyStopping
from datetime import *
from tensorflow.keras import optimizers
from keras.backend import int_shape
pd.set_option('display.max_colwidth',25)
from keras import backend as K
#from tensorflow.keras.layers import concatenate
#K.clear_session()

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

import pathlib
rando_seed = 1234
np.random.seed(rando_seed)
#os.chdir('/media/alex/easystore/ids_monthly/BLM_images/2014_11_12')
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report

2023-01-20 00:22:46.461854: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/slurm/lib64:
2023-01-20 00:22:46.461881: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [ ]:
#Evaluation of Model - Confusion Matrix Plot
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [ ]:
#cols = ['tweet_id', 'date', 'text', 'support', 'hashtags', 'users', 'urls', 'media_urls', 'nfollowers', 'nfriends', 'file_name', 'path']

#DATA = pd.read_csv('./meta_image_dataset_2020.csv', dtype = {'doc_id':str, 'picfiles':str,'label':int})
DATA = pd.read_csv('./elections_june21.csv', dtype = {'doc_id':str, 'picfiles':str,'label':str})
DATA.head()

In [ ]:
DATA.dtypes

In [ ]:
DATA['label'] = DATA['label'].map(str)
DATA.dtypes

In [ ]:
len(DATA)


In [ ]:
DATA.groupby('label').count()
# pie chart / bar chart to visualize data

In [ ]:
DATA = DATA.dropna(subset=['label'])
len(DATA)

In [ ]:
# must be divisible by batch size (50)

remove_n = 31
drop_indices = np.random.choice(DATA.index, remove_n, replace=False)
DATA = DATA.drop(drop_indices)
len(DATA)



In [ ]:
#Shuffle data

DATA = DATA.sample(frac=1, random_state=1234).reset_index(drop=True)
DATA.head()

In [ ]:
# Validation data used to check for overfitting

val_data = DATA.sample(n=500, replace=False, weights=None, random_state=1234, axis=None)
DATA = DATA.drop(val_data.index)
len(val_data)

In [ ]:
val_data.groupby('label').count()

In [ ]:
test_data = DATA.sample(n=500, replace=False, weights=None, random_state=1234, axis=None)
#test_data = test_data.sample(n=50, replace=False, weights=None, random_state=1234, axis=None)


In [ ]:
test_data.groupby('label').count()

In [ ]:
train_data = DATA.drop(test_data.index)
display("Length of val data:", len(val_data))
train_data.head()
train_data.groupby('label').count()

In [ ]:
# pie charts for each data set

df = pd.DataFrame()

train_ls = [train_data['label'].value_counts()]
test_ls = [test_data['label'].value_counts()]
val_ls = [val_data['label'].value_counts()]

df['train'] = train_ls[0]
df['val'] = val_ls[0]
df['test'] = test_ls[0]


df




In [ ]:
s = test_data['label'].value_counts()
s.plot(kind='pie', title="Test Set", figsize=(5, 5), autopct='%1.1f%%')

In [ ]:
#os.chdir('/media/alex/easystore/ids_monthly/BLM_images/apsa_sample3')

In [ ]:
# re: generators with multiple inputs

#trainer_path = 'train/'
#im_size = 28
img_width, img_height = 96, 96
batch_size = 50

input_imgen = ImageDataGenerator(rescale = 1./255)


def text_generator(a,labs, n):
    while True:
         for i in range(a.shape[0] // n):
            d2 = a[n*i:n*(i+1)]
            y_text = labs[n*i:n*(i+1)]
            yield d2, y_text       
            
def multi_input_generator(df, x_image, y_image, x_txt, y_txt, b_size): 
    t1 = input_imgen.flow_from_dataframe(
        dataframe=df,
        directory = './labim_all',
        x_col = x_image,
        y_col = y_image,
        target_size=(img_width, img_height),
        batch_size=b_size,
        class_mode='categorical',
        validate_filenames=False)
    
    t2 = text_generator(a=x_txt, labs = y_txt, n=b_size)
    
    while True:
        d1,y = t1.next()
        d1 = np.expand_dims(d1, axis = 0)
        d2, y_text = t2.__next__()
        yield [d2, d1[0]], [y_text, y]



In [ ]:
##############################################################
###### TEXT PRE-PROCESSING ###############################


# PARAMETERS
#MAX_SEQUENCE_LENGTH = 30
MAX_NUM_WORDS = 25
EMBEDDING_DIM = 100
#VALIDATION_SPLIT = 0.2


# SUBSET TO ONLY DATA AND LABELS
train_data_text = train_data[['features','label']]
train_labels_text = train_data['label'].values
train_data_text.head()

val_data_text = val_data[['features','label']]
val_labels_text = val_data['label'].values
val_data_text.head()

test_data_text = test_data[['features','label']]
test_labels_text = test_data['label'].values
test_data_text.head()

In [ ]:
# tokenizers for training text data
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS, lower=True,split=' ')
tokenizer.fit_on_texts(train_data_text['features'].values)
sequences = tokenizer.texts_to_sequences(train_data_text['features'].values)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

train_data_text = pad_sequences(sequences)
train_labels_text = ku.to_categorical(np.asarray(train_labels_text))

print('Shape of train data tensor:', train_data_text.shape)
print('Shape of train label tensor:', train_labels_text.shape)


##########################################################################
# tokenizers for val text data


#tokenizer = Tokenizer(num_words=MAX_NUM_WORDS, lower=True,split=' ')
#tokenizer.fit_on_texts(val_data_text['text'].values)
val_sequences = tokenizer.texts_to_sequences(val_data_text['features'].values)

word_index = tokenizer.word_index
print('Found %s unique val tokens.' % len(word_index))

val_data_text = pad_sequences(val_sequences)
val_labels_text = ku.to_categorical(np.asarray(val_labels_text))

#print('Shape of val data tensor:', val_data_text.shape)
#print('Shape of val label tensor:', val_labels_text.shape)

##########################################################################
# tokenizers for test text data


#tokenizer = Tokenizer(num_words=MAX_NUM_WORDS, lower=True,split=' ')
#tokenizer.fit_on_texts(val_data_text['text'].values)
test_sequences = tokenizer.texts_to_sequences(test_data_text['features'].values)

word_index = tokenizer.word_index
print('Found %s unique val tokens.' % len(word_index))

test_data_text = pad_sequences(test_sequences)
test_labels_text = ku.to_categorical(np.asarray(test_labels_text))

display('Shape of test data tensor:', test_data_text.shape)
display('Shape of test label tensor:', test_labels_text.shape)

In [ ]:
print('Shape of val data tensor:', val_data_text.shape)
print('Shape of val label tensor:', val_labels_text.shape)

In [ ]:
len(train_data_text[1])

In [ ]:
train_generator = multi_input_generator(df = train_data, x_image='picfiles', y_image='label',
                                    x_txt=train_data_text, y_txt=train_labels_text, b_size=batch_size)

val_generator = multi_input_generator(df = val_data, x_image='picfiles', y_image='label',
                                    x_txt=val_data_text, y_txt=val_labels_text, b_size=batch_size)

test_generator = multi_input_generator(df = test_data, x_image='picfiles', y_image='label',
                                    x_txt=test_data_text, y_txt=test_labels_text, b_size=batch_size)


In [ ]:
trainsetsize = len(train_data)
display('Number of training observations: ' + str(trainsetsize))

In [ ]:
valsetsize = len(val_data)
display('Number of val observations: ' + str(valsetsize))

testsetsize = len(test_data)
display('Number of test observations: ' + str(testsetsize))

In [ ]:

# Loading embeddings from Stanford NLP
# Good rundown of source code: https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html

# First, build index mapping words in the embeddings set
# to their embedding vector

print("Loading embeddings.")

embeddings_index = dict()
#texts = []
f = open('glove.6B.100d.txt', errors='ignore')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
#    texts.append(word)
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))





In [ ]:
# At this point we can leverage our embedding_index dictionary and our word_index to compute our embedding matrix:


print('Preparing embedding matrix.')

# prepare embedding matrix
num_words = min(MAX_NUM_WORDS, len(word_index)) + 1
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i > MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=25,
                            trainable=False)



In [ ]:
from tensorflow.keras.applications import inception_v3

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16

In [ ]:
# Load image model

image_model = VGG16(include_top=False, input_shape = (img_width, img_height, 3), weights='imagenet', pooling=None, classes=2)

for layer in image_model.layers:
    layer.trainable = False

In [ ]:
###########################################################################
########## MODEL CONSTRUCTION #############################################
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dot


sequence_input = Input(shape=(None, ), dtype='int64')
embedded_sequences = embedding_layer(sequence_input)
x_text = Conv1D(32, kernel_size = 5, activation="relu", padding = 'same')(embedded_sequences)
x_text = MaxPooling1D(5)(x_text)
x_text = Conv1D(32, kernel_size = 5, activation="relu", padding = 'same')(x_text)
x_text = GlobalMaxPooling1D()(x_text)
x_text = Dense(16, activation="relu")(x_text)
#x_text = Dropout(0.5)(x_text)
preds = Dense(2, activation='relu')(x_text)



# Adding custom Layers

x_image = image_model.output

x_image = Conv2D(3, 1, activation="relu", padding = 'same')(x_image)
x_image = MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='same')(x_image)
x_image = Conv2D(3, 1, activation="relu", padding = 'same')(x_image)
x_image = MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='same')(x_image)
x_image = Conv2D(3, 1, activation="relu", padding = 'same')(x_image)
x_image = MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='same')(x_image)
x_image = Flatten()(x_image)
img_predictions = Dense(2, activation="relu")(x_image)


#merged = Dot(axes=1)([preds, img_predictions])

merged = Concatenate()([preds, img_predictions])

# We stack densely-connected network on top
x = Dense(64, activation='relu')(merged)
x = Dense(32, activation='relu')(x)
x = Dense(16, activation='relu')(x)
x = Dense(8, activation='relu')(x)
x = Dense(4, activation='relu')(x)
#x = Dropout(0.5)(x)
main_output = Dense(2, activation='sigmoid', name = 'main_output')(x)



# Defining a model with two inputs and one outputs
elections_model = Model([sequence_input, image_model.input], [main_output])


tbCallBack = TensorBoard(log_dir='./Graph', histogram_freq=0, write_graph=True, write_images=False)


print("ROMA VICTOR!")

sgd = optimizers.SGD(lr=0.01, decay=1e-5, momentum=0.95, nesterov=True, clipvalue=.5)
elections_model.compile(optimizer=sgd, loss="categorical_crossentropy", metrics = ['categorical_accuracy'])




In [ ]:
elections_model.summary()

In [ ]:
history = elections_model.fit(train_generator,
          steps_per_epoch=trainsetsize//batch_size,
                    class_weight = None,
                        epochs = 50,
                        validation_data = val_generator,
                        validation_steps = valsetsize//batch_size)
                        #use_multiprocessing=True,
                        #shuffle=False)
        



In [ ]:
training_metrics_ti = pd.read_csv('training_metrics.csv')
training_metrics_ti

In [ ]:
acc = training_metrics_ti['accuracy']
val_acc = training_metrics_ti['val_accuracy']

loss = training_metrics_ti['loss']
val_loss = training_metrics_ti['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')

plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')

#plt.savefig(train_file_name)
plt.show()

In [ ]:
#import pickle        
# create an iterator object with write permission - model.pkl


#elections_model.save('elections_model916.h5')  # creates a HDF5 file 'my_model.h5'


In [2]:
model1 = load_model('elections_model916.h5')
model1.summary()

2023-01-20 00:25:09.829000: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/slurm/lib64:
2023-01-20 00:25:09.829031: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-01-20 00:25:09.829044: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (gl3185.arc-ts.umich.edu): /proc/driver/nvidia/version does not exist
2023-01-20 00:25:09.829681: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 96, 96, 3)]  0           []                               
                                                                                                  
 block1_conv1 (Conv2D)          (None, 96, 96, 64)   1792        ['input_1[0][0]']                
                                                                                                  
 block1_conv2 (Conv2D)          (None, 96, 96, 64)   36928       ['block1_conv1[0][0]']           
                                                                                                  
 block1_pool (MaxPooling2D)     (None, 48, 48, 64)   0           ['block1_conv2[0][0]']           
                                                                                            

In [ ]:
test_generator1 = multi_input_generator(df = test_data, x_image='picfiles', y_image='label',
                                    x_txt=test_data_text, y_txt=test_labels_text, b_size=batch_size)

"""
for i, tweet in enumerate(test_generator1):
        
        print("Image Number: ", i)
    
        print(img)
        
        if i >= n:
            break


"""

In [ ]:
n = 499
train_labels0 = []
train_labels1 = []
train_preds = []


for i, tweet in enumerate(train_generator):
    
    if i % 10 == 0:
            
        print("Image Number: ", i)

    pred1 = model1.predict(tweet[0], use_multiprocessing = False)

    train_preds.append(pred1)

    train_labels0.append(tweet[1][0])
    train_labels1.append(tweet[1][1])

    if i >= n:
        break

In [ ]:
n = 499
labels0 = []
labels1 = []
preds = []


for i, tweet in enumerate(test_generator):
    
    if i % 10 == 0:
            
        print("Image Number: ", i)

    pred1 = model1.predict(tweet[0], use_multiprocessing = False)

    preds.append(pred1)

    labels0.append(tweet[1][0])
    labels1.append(tweet[1][1])

    if i >= n:
        break

In [ ]:
preds

In [ ]:

train_predictions = []
# Convert to an array of binary values

for i in train_preds:
    
    train_predictions.append(np.argmax(i, axis=1))
    
len(predictions)

In [ ]:
train_predictions

In [ ]:
train_labs0 = []

for i in train_labels0:
    
    train_labs0.append(np.argmax(i, axis=1))


train_labs0[0:10]

In [ ]:
train_labs1 = []

for i in train_labels1:
    
    train_labs1.append(np.argmax(i, axis=1))


train_labs1[0:10]

In [ ]:
train_predictions_final = np.concatenate(train_predictions).ravel().tolist()
train_labs0_final = np.concatenate(train_labs0).ravel().tolist()


In [ ]:
train_labs1_final = np.concatenate(train_labs1).ravel().tolist()

In [ ]:
train_predictions_final

In [ ]:
import itertools

# Grab true labels from the validation generator


# Generate confusion matrix

# labels = ['W', 'NW']
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html


conf_mat = confusion_matrix(train_labs0_final, train_predictions_final, labels = [0, 1], sample_weight=None)
#print(conf_mat)

plot_confusion_matrix(conf_mat, classes=[0, 1],
                      title='Confusion matrix, with normalization', normalize = True)

In [ ]:
from keras import backend as K
K.clear_session()